In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# 필수 라이브러리 불러오기
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# missingvalue 모듈이 없을 경우 내부에 함수 직접 정의
try:
    from missingvalue import (
        filling_zero, filling_minus1, filling_mean,
        filling_median, filling_cluster_mean, filling_knn
    )
except ModuleNotFoundError:
    def filling_zero(df):
        df = df.copy()
        df.fillna(0, inplace=True)
        return df

    def filling_minus1(df):
        df = df.copy()
        df.fillna(-1, inplace=True)
        return df

    def filling_mean(df):
        df = df.copy()
        for col in df.select_dtypes(include=["number"]):
            df[col] = df[col].fillna(df[col].mean())
        return df

    def filling_median(df):
        df = df.copy()
        for col in df.select_dtypes(include=["number"]):
            df[col] = df[col].fillna(df[col].median())
        return df

    def filling_cluster_mean(df):
        df = df.copy()
        if 'cluster' not in df.columns:
            raise ValueError("'cluster' column missing in dataframe")
        for col in df.select_dtypes(include=["number"]):
            if col == 'cluster':
                continue
            for cl in df['cluster'].unique():
                mean_val = df.loc[df['cluster'] == cl, col].mean()
                df.loc[(df['cluster'] == cl) & (df[col].isnull()), col] = np.array(mean_val, dtype=df[col].dtype)
        return df

    from sklearn.impute import KNNImputer
    def filling_knn(df):
        df = df.copy()
        numeric_cols = df.select_dtypes(include=["number"]).columns
        imputer = KNNImputer(n_neighbors=5)
        df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
        return df

    def filling_mode(df):
        df = df.copy()
        for col in df.columns:
            if df[col].isnull().sum() > 0:
                df[col] = df[col].fillna(df[col].mode()[0])
        return df

    def filling_forward(df):
        df = df.copy()
        df = df.ffill()
        return df

    def filling_backward(df):
        df = df.copy()
        df = df.bfill()
        return df

    def filling_interpolate(df):
        df = df.copy()
        num_cols = df.select_dtypes(include=["number"]).columns
        df[num_cols] = df[num_cols].interpolate(method='linear', limit_direction='both')
        return df

    def filling_random_sample(df):
        df = df.copy()
        for col in df.columns:
            if df[col].isnull().sum() > 0:
                non_null_values = df[col].dropna().values
                fill_values = np.random.choice(non_null_values, size=df[col].isnull().sum(), replace=True)
                df.loc[df[col].isnull(), col] = fill_values
        return df

    def filling_constant(df, constant=-999):
        df = df.copy()
        df.fillna(constant, inplace=True)
        return df

base_path = "/content/drive/MyDrive/SC/Hackathon/기업 성공 확률 예측/open/train.csv"
target_col = "성공확률"
save_results = {}

original_df = pd.read_csv(base_path)

from sklearn.cluster import KMeans

# 클러스터링을 통해 기업을 그룹화하여 cluster_mean 전략에 사용

def assign_clusters(df, n_clusters=5):
    df = df.copy()
    numeric_cols = ['직원 수', '총 투자금(억원)', '고객수(백만명)', '기업가치(백억원)']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').str.extract(r"(\d+\.?\d*)")[0], errors='coerce')
    km = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['cluster'] = km.fit_predict(df[numeric_cols].fillna(df[numeric_cols].mean()))
    return df

clustered_df = assign_clusters(original_df)

# 전략 이름과 결측치 처리 함수 매핑
strategies = {
    "zero": filling_zero,
    "minus1": filling_minus1,
    "mean": filling_mean,
    "median": filling_median,
    "cluster_mean": filling_cluster_mean,
    "knn": filling_knn,
    "mode": filling_mode,
    "forward": filling_forward,
    "backward": filling_backward,
    "interpolate": filling_interpolate,
    "random_sample": filling_random_sample,
    "constant": filling_constant,
}

# 각 결측치 처리 전략에 대해 모델 성능(MAE) 평가
for name, strategy_fn in strategies.items():
    print(f"전략: {name} 평가 중...")
    try:
        df = strategy_fn(clustered_df)
        # 타겟 및 식별자 컬럼 제거 후 수치형 피처만 선택
        X = df.drop(columns=[target_col, '기업번호'], errors='ignore')
        y = df[target_col]
        X = X.select_dtypes(include=[np.number])

        # 5-Fold 교차검증 설정
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        maes = []

        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=-1)
            model.fit(X_train, y_train)
            pred = model.predict(X_val)
            mae = mean_absolute_error(y_val, pred)
            maes.append(mae)

        avg_mae = np.mean(maes)
        save_results[name] = avg_mae
        print(f"{name} 평균 MAE: {avg_mae:.6f}")

    except Exception as e:
        print(f"전략 {name} 실패: {e}")

# 모든 전략에 대한 MAE 출력
print("전체 MAE 결과 요약:")

for name, mae in save_results.items():
    print(f" - {name}: MAE = {mae:.6f}")

전략: zero 평가 중...
zero 평균 MAE: 0.204596
전략: minus1 평가 중...
minus1 평균 MAE: 0.204596
전략: mean 평가 중...
mean 평균 MAE: 0.205048
전략: median 평가 중...
median 평균 MAE: 0.205764
전략: cluster_mean 평가 중...
cluster_mean 평균 MAE: 0.205314
전략: knn 평가 중...
knn 평균 MAE: 0.205037
전략: mode 평가 중...
mode 평균 MAE: 0.205554
전략: forward 평가 중...
forward 평균 MAE: 0.204825
전략: backward 평가 중...
backward 평균 MAE: 0.204346
전략: interpolate 평가 중...
interpolate 평균 MAE: 0.205720
전략: random_sample 평가 중...
random_sample 평균 MAE: 0.205188
전략: constant 평가 중...
constant 평균 MAE: 0.204596
전체 MAE 결과 요약:
 - zero: MAE = 0.204596
 - minus1: MAE = 0.204596
 - mean: MAE = 0.205048
 - median: MAE = 0.205764
 - cluster_mean: MAE = 0.205314
 - knn: MAE = 0.205037
 - mode: MAE = 0.205554
 - forward: MAE = 0.204825
 - backward: MAE = 0.204346
 - interpolate: MAE = 0.205720
 - random_sample: MAE = 0.205188
 - constant: MAE = 0.204596


| 순위 | 버전 | 전략            | MAE       | 처리 방식 설명                           |
|------|------|------------------|-----------|-------------------------------------------|
| 1    | I    | backward         | 0.204346  | 뒤의 값으로 채움 (backward fill)         |
| 2    | A    | zero             | 0.204596  | 모든 결측치를 0으로 대체                 |
| 2    | D    | minus1           | 0.204596  | 모든 결측치를 -1로 대체                  |
| 2    | L    | constant         | 0.204596  | 고정값(-999)으로 대체                    |
| 5    | H    | forward          | 0.204825  | 앞의 값으로 채움 (forward fill)          |
| 6    | F    | knn              | 0.205037  | KNN 보간으로 대체                        |
| 7    | B    | mean             | 0.205048  | 모든 결측치를 평균값으로 대체            |
| 8    | E    | cluster_mean     | 0.205314  | 클러스터별 평균값으로 대체               |
| 9    | G    | mode             | 0.205554  | 최빈값으로 대체                          |
| 10   | C    | median           | 0.205764  | 모든 결측치를 중앙값으로 대체            |
| 11   | J    | interpolate      | 0.205720  | 선형 보간법으로 대체                     |
| 12   | K    | random_sample    | 0.206227  | 열의 기존 값 중 무작위 샘플로 대체       |


### 결측치 비율

In [18]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/SC/Hackathon/기업 성공 확률 예측/open/train.csv")

missing_ratio = df.isnull().mean().reset_index()
missing_ratio.columns = ['컬럼명', '결측치 비율']
missing_ratio = missing_ratio[missing_ratio['결측치 비율'] > 0].sort_values('결측치 비율', ascending=False)

missing_ratio

,컬럼명,결측치 비율
8,고객수(백만명),0.301645
12,기업가치(백억원),0.278793
3,분야,0.195841
5,직원 수,0.039762


In [19]:
# 필수 라이브러리 불러오기
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# missingvalue 모듈이 없을 경우 내부에 함수 직접 정의
try:
    from missingvalue import (
        filling_zero, filling_minus1, filling_mean,
        filling_median, filling_cluster_mean, filling_knn
    )
except ModuleNotFoundError:
    def filling_zero(df):
        df = df.copy()
        df.fillna(0, inplace=True)
        return df

    def filling_minus1(df):
        df = df.copy()
        df.fillna(-1, inplace=True)
        return df

    def filling_mean(df):
        df = df.copy()
        for col in df.select_dtypes(include=["number"]):
            df[col] = df[col].fillna(df[col].mean())
        return df

    def filling_median(df):
        df = df.copy()
        for col in df.select_dtypes(include=["number"]):
            df[col] = df[col].fillna(df[col].median())
        return df

    def filling_cluster_mean(df):
        df = df.copy()
        if 'cluster' not in df.columns:
            raise ValueError("'cluster' column missing in dataframe")
        for col in df.select_dtypes(include=["number"]):
            if col == 'cluster':
                continue
            for cl in df['cluster'].unique():
                mean_val = df.loc[df['cluster'] == cl, col].mean()
                df.loc[(df['cluster'] == cl) & (df[col].isnull()), col] = np.array(mean_val, dtype=df[col].dtype)
        return df

    from sklearn.impute import KNNImputer
    def filling_knn(df):
        df = df.copy()
        numeric_cols = df.select_dtypes(include=["number"]).columns
        imputer = KNNImputer(n_neighbors=5)
        df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
        return df

    def filling_mode(df):
        df = df.copy()
        for col in df.columns:
            if df[col].isnull().sum() > 0:
                df[col] = df[col].fillna(df[col].mode()[0])
        return df

    def filling_forward(df):
        df = df.copy()
        df = df.ffill()
        return df

    def filling_backward(df):
        df = df.copy()
        df = df.bfill()
        return df

    def filling_interpolate(df):
        df = df.copy()
        num_cols = df.select_dtypes(include=["number"]).columns
        df[num_cols] = df[num_cols].interpolate(method='linear', limit_direction='both')
        return df

    def filling_random_sample(df):
        df = df.copy()
        for col in df.columns:
            if df[col].isnull().sum() > 0:
                non_null_values = df[col].dropna().values
                fill_values = np.random.choice(non_null_values, size=df[col].isnull().sum(), replace=True)
                df.loc[df[col].isnull(), col] = fill_values
        return df

    def filling_constant(df, constant=-999):
        df = df.copy()
        df.fillna(constant, inplace=True)
        return df

base_path = "/content/drive/MyDrive/SC/Hackathon/기업 성공 확률 예측/open/train.csv"
target_col = "성공확률"
save_results = {}

original_df = pd.read_csv(base_path)

from sklearn.cluster import KMeans

# 클러스터링을 통해 기업을 그룹화하여 cluster_mean 전략에 사용

def assign_clusters(df, n_clusters=5):
    df = df.copy()
    numeric_cols = ['직원 수', '총 투자금(억원)', '고객수(백만명)', '기업가치(백억원)']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').str.extract(r"(\d+\.?\d*)")[0], errors='coerce')
    km = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['cluster'] = km.fit_predict(df[numeric_cols].fillna(df[numeric_cols].mean()))
    return df

clustered_df = assign_clusters(original_df)

# 전략 이름과 결측치 처리 함수 매핑
strategies = {
    "zero": filling_zero,
    "minus1": filling_minus1,
    "mean": filling_mean,
    "median": filling_median,
    "cluster_mean": filling_cluster_mean,
    "knn": filling_knn,
    "mode": filling_mode,
    "forward": filling_forward,
    "backward": filling_backward,
    "interpolate": filling_interpolate,
    "random_sample": filling_random_sample,
    "constant": filling_constant,
}

# 각 결측치 처리 전략에 대해 모델 성능(MAE) 평가
# 컬럼별 전략 × 처리 방법 전체 실험
columns_to_test = [
    "고객수(백만명)",
    "기업가치(백억원)",
    "분야",
    "직원 수"
]

for col in columns_to_test:
    for strat_name, strat_func in strategies.items():
        df = clustered_df.copy()
        try:
            if col not in df.columns:
                continue
            modified = strat_func(df[[col, 'cluster']] if 'cluster' in df.columns else df[[col]])
            df[col] = modified[col]

            X = df.drop(columns=[target_col, '기업번호'], errors='ignore')
            y = df[target_col]
            X = X.select_dtypes(include=[np.number])

            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            maes = []

            for train_idx, val_idx in kf.split(X):
                X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

                model = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=-1)
                model.fit(X_train, y_train)
                pred = model.predict(X_val)
                mae = mean_absolute_error(y_val, pred)
                maes.append(mae)

            avg_mae = np.mean(maes)
            key = f"{col}_{strat_name}"
            save_results[key] = avg_mae
            print(f"{key} 평균 MAE: {avg_mae:.6f}")

        except Exception as e:
            print(f"{col}_{strat_name} 실패: {e}")

# 모든 전략에 대한 MAE 출력
print("전체 MAE 결과 요약:")

for name, mae in save_results.items():
    print(f" - {name}: MAE = {mae:.6f}")

고객수(백만명)_zero 평균 MAE: 0.204994
고객수(백만명)_minus1 평균 MAE: 0.204994
고객수(백만명)_mean 평균 MAE: 0.205594
고객수(백만명)_median 평균 MAE: 0.205119
고객수(백만명)_cluster_mean 평균 MAE: 0.204840
고객수(백만명)_knn 평균 MAE: 0.205326
고객수(백만명)_mode 평균 MAE: 0.204547
고객수(백만명)_forward 평균 MAE: 0.204599
고객수(백만명)_backward 평균 MAE: 0.205709
고객수(백만명)_interpolate 평균 MAE: 0.204420
고객수(백만명)_random_sample 평균 MAE: 0.205802
고객수(백만명)_constant 평균 MAE: 0.204994
기업가치(백억원)_zero 평균 MAE: 0.205618
기업가치(백억원)_minus1 평균 MAE: 0.205618
기업가치(백억원)_mean 평균 MAE: 0.205240
기업가치(백억원)_median 평균 MAE: 0.205529
기업가치(백억원)_cluster_mean 평균 MAE: 0.205143
기업가치(백억원)_knn 평균 MAE: 0.205926
기업가치(백억원)_mode 평균 MAE: 0.205312
기업가치(백억원)_forward 평균 MAE: 0.205756
기업가치(백억원)_backward 평균 MAE: 0.205035
기업가치(백억원)_interpolate 평균 MAE: 0.205671
기업가치(백억원)_random_sample 평균 MAE: 0.205543
기업가치(백억원)_constant 평균 MAE: 0.205618
분야_zero 평균 MAE: 0.204881
분야_minus1 평균 MAE: 0.204881
분야_mean 평균 MAE: 0.204881
분야_median 평균 MAE: 0.204881
분야_cluster_mean 평균 MAE: 0.204881
분야_knn 평균 MAE: 0.204881
분야_mode

| 컬럼명         | 컬럼 내 순위 | 버전               | MAE       |
|----------------|--------------|--------------------|-----------|
| 고객수(백만명)  | 1            | J(interpolate)     | 0.204420  |
| 고객수(백만명)  | 2            | G(mode)            | 0.204547  |
| 고객수(백만명)  | 3            | H(forward)         | 0.204599  |
| 고객수(백만명)  | 4            | E(cluster_mean)    | 0.204840  |
| 고객수(백만명)  | 5            | A(zero)            | 0.204994  |
| 고객수(백만명)  | 5            | D(minus1)          | 0.204994  |
| 고객수(백만명)  | 5            | L(constant)        | 0.204994  |
| 고객수(백만명)  | 8            | knn(F)             | 0.205326  |
| 고객수(백만명)  | 9            | B(mean)            | 0.205594  |
| 고객수(백만명)  | 10           | C(median)          | 0.205119  |
| 고객수(백만명)  | 11           | I(backward)        | 0.205709  |
| 고객수(백만명)  | 12           | K(random_sample)   | 0.205802  |
| 기업가치(백억원) | 1            | I(backward)        | 0.205035  |
| 기업가치(백억원) | 2            | E(cluster_mean)    | 0.205143  |
| 기업가치(백억원) | 3            | B(mean)            | 0.205240  |
| 기업가치(백억원) | 4            | G(mode)            | 0.205312  |
| 기업가치(백억원) | 5            | C(median)          | 0.205529  |
| 기업가치(백억원) | 6            | K(random_sample)   | 0.205543  |
| 기업가치(백억원) | 7            | J(interpolate)     | 0.205671  |
| 기업가치(백억원) | 8            | H(forward)         | 0.205756  |
| 기업가치(백억원) | 9            | knn(F)             | 0.205926  |
| 기업가치(백억원) | 10           | A(zero)            | 0.205618  |
| 기업가치(백억원) | 10           | D(minus1)          | 0.205618  |
| 기업가치(백억원) | 10           | L(constant)        | 0.205618  |
| 분야           | 1            | 모든 전략 동일       | 0.204881  |
| 직원 수         | 1            | J(interpolate)     | 0.203725  |
| 직원 수         | 2            | I(backward)        | 0.204438  |
| 직원 수         | 3            | H(forward)         | 0.204533  |
| 직원 수         | 4            | G(mode)            | 0.204562  |
| 직원 수         | 5            | A(zero)            | 0.204289  |
| 직원 수         | 5            | D(minus1)          | 0.204289  |
| 직원 수         | 5            | L(constant)        | 0.204289  |
| 직원 수         | 8            | E(cluster_mean)    | 0.205237  |
| 직원 수         | 9            | K(random_sample)   | 0.205447  |
| 직원 수         | 10           | B(mean)            | 0.205407  |
| 직원 수         | 10           | C(median)          | 0.205407  |
| 직원 수         | 12           | knn(F)             | 0.205831  |


In [21]:
import pandas as pd
import numpy as np

# 테스트용 DataFrame 구성
df = pd.read_csv("/content/drive/MyDrive/SC/Hackathon/기업 성공 확률 예측/open/train.csv")

columns_to_test = ["고객수(백만명)", "기업가치(백억원)", "분야", "직원 수"]

def filling_mode(df):
    df = df.copy()
    changes = {}
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            before = df[col].isnull().sum()
            df[col] = df[col].fillna(df[col].mode()[0])
            after = df[col].isnull().sum()
            changes[col] = before - after
    return df, changes

results = []
for col in columns_to_test:
    subset = df[[col]].copy()
    # 결측치 강제 삽입 (실험용) → 안 해도 됨
    # subset.loc[subset.sample(frac=0.1).index] = np.nan

    filled, changes = filling_mode(subset)
    results.append({
        '컬럼명': col,
        '처리 전 결측치': df[col].isnull().sum(),
        '처리 후 결측치': filled[col].isnull().sum(),
        '채워진 수': changes.get(col, 0)
    })

pd.DataFrame(results)

,컬럼명,처리 전 결측치,처리 후 결측치,채워진 수
0,고객수(백만명),1320,0,1320
1,기업가치(백억원),1220,0,1220
2,분야,857,0,857
3,직원 수,174,0,174
